In [ ]:
import pandas as pd
import geopandas as gpd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio

# Assuming your GeoDataFrame is named 'gdf' and already in long format
# with columns: GEOID, acs_year, indexed_vulnerability, geometry, etc.
eaz = gpd.read_file('atx_eaz_multi_year.geojson')

# Convert to GeoJSON for Plotly
geojson = eaz.__geo_interface__

In [ ]:
# Creating Plotly figure with subplots
# Create subplots
fig = make_subplots(
    rows=1, cols=2,
    specs=[[{'type': 'choroplethmapbox'}, {'type': 'bar'}]],
    column_widths=[1.0, 0.55]
)

# Defining the order of eaz_type categories
order = ['Least Vulnerable', 'Medium-Low Vulnerable', 'Medium Vulnerable', 'Medium-High Vulnerable', 'Most Vulnerable']
order_mapping = {name: i for i, name in enumerate(order)}

# Adding traces for the plots
for year in eaz['acs_year'].unique():
    year_data = eaz[eaz['acs_year'] == year]

    # Chloropleth traces
    if year == 2019:    # Starting year
        hovertemplate = (
            "<b>%{customdata[0]}</b><br>" +
            "<b>Year:</b> %{customdata[1]}<br>" +
            "<b>Vulnerability Index:</b> %{z:.2f}<br>" +
            "<extra></extra>"
        )
    else:
        hovertemplate = (
            "<b>%{customdata[0]}</b><br>" +
            "<b>Year:</b> %{customdata[1]}<br>" +
            "<b>Vulnerability Index:</b> %{z:.2f}<br>" +
            "<b>Difference from 2019:</b> %{customdata[2]:+.2f}<br>" +
            "<extra></extra>"
        )

    # Customdata formatted for hover information
    customdata = year_data[['NAME', 'acs_year', 'difference_since_2019']].copy()
    customdata['difference_since_2019'] = customdata['difference_since_2019'].apply(
        lambda x: f"{x:+.2f}" if pd.notnull(x) else "N/A"
    )
    
    fig.add_trace(go.Choroplethmapbox(
        geojson=geojson,
        locations=year_data.index,
        z=year_data['indexed_vulnerability'],
        # colorscale="RdYlGn_r",
        colorscale="Reds",
        zmin=eaz['indexed_vulnerability'].min(),
        zmax=eaz['indexed_vulnerability'].max(),
        marker_opacity=0.8,
        marker_line_width=0.25,
        visible=(year == eaz['acs_year'].min()),  # Only the first year visible initially
        name=str(year),  # Add name for the slider
        hovertemplate=hovertemplate,
        customdata=customdata[['NAME', 'acs_year', 'difference_since_2019']],
        colorbar=dict(
            x = 0.1,
            y = -0.015,
            thickness=10,
            outlinecolor='rgba(0,0,0,0)',  # Transparent outline
            bgcolor='rgba(255,255,255,1)', # White background
            len = 0.2,
            orientation = 'h'
        )
    ), row=1, col=1)


    # Bar graph traces
    bar_data = year_data[['eaz_type']].value_counts().reset_index()
    bar_data.columns = ['eaz_type', 'count']
    bar_data['order'] = bar_data['eaz_type'].map(order_mapping)
    bar_data = bar_data.sort_values('order')  # Sort by the specified order

    fig.add_trace(go.Bar(
        x=bar_data['eaz_type'],
        y=bar_data['count'],
        marker=dict(color='red'),
        showlegend=False,
        visible=(year == eaz['acs_year'].min()),
        hovertemplate=(
            "<b>Number of census tracts:</b> %{y}<br>" +
            "<extra></extra>"
        )
    ), row=1, col=2)


# Update layout
steps = []
for year in eaz['acs_year'].unique():

    visible = []

    for y in eaz['acs_year'].unique():
        visible.append(y == year)
        visible.append(y == year)

    step = dict(
        label=str(year),
        method="update",
        args=[
            {
                "visible": visible, 
                "title": f"Indexed Vulnerability - {year}"
            }
        ],
    )
    steps.append(step)

fig.update_layout(
    title_text="Equity Analysis Zones<br><sup>Indexed Vulnerability Score</sup>",
    title_font=dict(size=25),
    title_x = 0.5,
    mapbox_style="carto-positron",
    width=1200,
    height=800,
    mapbox_zoom=8,
    mapbox_center={"lat": 30.2672, "lon": -97.7431},
    sliders=[dict(
        active=0,
        steps=steps,
        currentvalue=dict(
            prefix="Selected year: ",
            font=dict(size=10, color="black"),
            visible=True,
            xanchor='left'
        )
    )]
)


# Customizing the look of the bar plot
fig.update_xaxes(
    tickmode='array',
    tickvals=[0, 1, 2, 3, 4],
    ticktext=['Least<br>Vulnerable', 'Medium-Low<br>Vulnerable', 'Medium<br>Vulnerable', 'Medium-High<br>Vulnerable', 'Most<br>Vulnerable'],
    tickfont=dict(size=9),
    tickangle=0, 
    row=1, col=2    # Apply to the bar plot
)

fig.update_yaxes(
    gridcolor='lightgray',
    # tickvals=[20, 40, 60, 80, 100, 120, 140, 160],
    row=1, col=2     # Apply to the bar plot
)

fig.update_layout(
    plot_bgcolor='rgba(0,0,0,0)'
)

# Show the figure
fig.show()

In [ ]:
# Exporting to HTML
fig.write_html("eaz_map.html")